<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-agents/blob/main/mcp_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI MCP Agent

https://platform.openai.com/logs?api=traces



In [ ]:
!pip install openai mcp python-dotenv anyio
!pip install openai-agents fastmcp nest_asyncio

In [ ]:
# set up imports

import requests
import nest_asyncio
import threading
import time
from fastmcp import FastMCP
from typing import Any
import httpx
import warnings

# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

nest_asyncio.apply()

# create server
mcp = FastMCP("Addition Server")

@mcp.tool()
def add_numbers(a : int, b : int) -> int:
    return a + b

# Define the function to run the server
def run_server():
    # Use transport="streamable-http" for compatibility with notebooks/Colab
    print("🚀 Starting FastMCP server in background thread...")
    mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(5)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")

In [ ]:
import os
from google.colab import userdata

# Initialize API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
import sys
import os
from agents import Agent, Runner
from agents.mcp import MCPServerStreamableHttp, MCPServerStreamableHttpParams

# Define the async function
async def main(input: str):
    name: str = "colab-server"
    params: MCPServerStreamableHttpParams = {
        "url": "http://localhost:8000/mcp"
    }

    async with MCPServerStreamableHttp(
        name=name,
        params=params,
        cache_tools_list=True,
        max_retry_attempts=3
    ) as server:
        agent = Agent(
            name="Assistant",
            instructions="You are a helpful assistant that can add numbers together using a MCP tool.",
            mcp_servers=[server]
        )

        result = await Runner.run(agent, input=input)
        print(result.final_output)

# Run the async function
await main("What is 5 plus 3")